In [225]:
import torch
import torch.nn as nn
import numpy as np
from torch.nn import CrossEntropyLoss
import torch.optim as optim
import pandas as pd
from torch.utils.data import TensorDataset, Dataset
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import torch.nn.functional as F


In [ ]:
# Khởi tạo Tensor
# Từ mảng 
data = [[1,2], [3,4]]
# Từ numpy array
data = np.array(([1,2], [3,4]), dtype=float)
x_data = torch.tensor(data)
print(x_data)
print()
shape = (2,3,)
rand_tensor = torch.rand(shape)
ones_tensor = torch.ones(shape)
zeros_tensor = torch.zeros(shape)
print(rand_tensor)
print(ones_tensor)
print(zeros_tensor)

In [ ]:
# Thông số của Tensor
tensor = torch.rand((3,4))

print(tensor.shape)
print(tensor.dtype)
print(tensor.device)


In [ ]:
# Phép toán với Tensor

device = 'cuda' if torch.cuda.is_available() else 'cpu'
tensor = tensor.to(device)
print(tensor.device)

# Chuyển từ numpy sang torch

n = np.ones(5)
t = torch.from_numpy(n)

# Chuyển từ torch sang numpy

t = torch.ones(5)
n = t.numpy()

# torch.autograd

Mạng NN là tập hợp các hàm lồng nhau để thực thi 1 tập input
Các hàm này được xác định bởi weights và bias - trọng số
Các trọng số được lưu dứoi dạng Tensor

Forward: Dự đoán kết quả đầu ra. Input -> L1 -> ... -> Output

Backward: Điều chỉnh trọng số ứng với sai số trong dự đoán
Duyệt ngược từ đầu ra, thu thập đạo hàm của sai số theo gradient, tối ưu hoá bằng giảm gradient

In [ ]:
# Activation Function

model = nn.Sequential(
    nn.Linear(10, 18),
    nn.Linear(18, 20),
    nn.Linear(20, 5),
    nn.Softmax(dim=-1)
)

a = torch.rand(10).reshape(-1,10)
output = model(a)
print(output)


# Các bài toán phân loại đơn giản

In [ ]:
# Binary Classification

input_data = torch.tensor(
    np.random.rand(5,6), dtype= torch.float
)
print(f'input data: ', input_data)
print()

model = nn.Sequential(
    nn.Linear(6,4),
    nn.Linear(4,1),
    nn.Sigmoid()
)

output = model(input_data)
print(f'output: ', output)
# five probs [0, 1] for 5 animals
# class = 1 mammals, class = 0 not mammals

In [ ]:
# Multi-class classification

n_classes = 3
model = nn.Sequential(
    nn.Linear(6,4),
    nn.Linear(4,n_classes),
    nn.Softmax(-1)
)

output = model(input_data)
print(f'output: ', [torch.argmax(i) for i in output])
print(f'output: ', output)

In [ ]:
# Regression
model = nn.Sequential(
    nn.Linear(6, 4), # First linear layer
    nn.Linear(4, 1) # Second linear layer
)

output = model(input_data)
print(output)

# Sử dụng Loss function

In [ ]:

# # CrossEntropyLoss

loss = CrossEntropyLoss(output, input_data)
print(loss)

loss.backward()


# Lan truyền ngược

In [ ]:


optimizer = optim.SGD(model.parameters(), lr=0.001)

# Perform parameters updates

optimizer.step()


# Load data

In [ ]:
import torch
from torch.utils.data import TensorDataset

np.random.seed(42)

X = np.random.uniform(low=0, high=2, size=(5,5)).astype(int)
X1 = (np.random.uniform(low=1,high=3,size=(5,1))).astype(int) * 2
X2 = np.random.uniform(low=0, high=2, size=(5,2)).astype(int)

X = np.concatenate((X,X1), axis=1)
X = np.concatenate((X,X2), axis=1)
print(X)

Y = [0, 0, 1, 1, 2]
print(Y)


dataset = TensorDataset(torch.tensor(X), torch.tensor(Y))
input_sample, label_sample = dataset[0]
print(f'input_sample: ', input_sample)
print(f'label_sample: ', label_sample)

In [ ]:
from torch.utils.data import DataLoader
batch_size = 2
shuffle = True
dataloader = DataLoader(dataset, batch_size=batch_size,shuffle=shuffle)

for batch_inputs, batch_labels in dataloader:
    print(f'batch_inputs: ', batch_inputs)
    print(f'batch_lables: ', batch_labels)

# Khởi tạo Layer

In [ ]:
# Khởi tạo Layer bình thường
layer = nn.Linear(64,128)
print(layer.weight.min(), layer.weight.max())

# Khởi tạo Layer uniform
layer = nn.Linear(64,128)
nn.init.uniform_(layer.weight)

print(layer.weight.min(), layer.weight.max())

# Reuser Model đã học

In [ ]:
model = nn.Linear(5,12)
# Lưu weight
torch.save(model.state_dict(), 'model.pth')

# Tạo cái vỏ mới
new_model = nn.Linear(5,12)
# Load weight
new_model.load_state_dict(torch.load('model.pth', weights_only=True))

print(torch.equal(new_model.weight, model.weight))
print(torch.equal(new_model.bias, model.bias))

# Fine Tuning
- Learning rate nhỏ
- Đóng băng 1 vài layer
- Đóng bằng layer gần input và fine-tune layer gần output

In [ ]:
model = nn.Sequential(
        nn.Linear(5, 10),
        nn.Linear(10, 5))

# Freeze weight layer 1
for name, param in model.named_parameters():
    if name == '0.weight':
        param.requires_grad = False
for name, param in model.named_parameters():
    print(f"{name}: requires_grad = {param.requires_grad}")

print()

# Freeze layer 1
for param in model[0].parameters():
    param.requires_grad = False
for name, param in model.named_parameters():
    print(f"{name}: requires_grad = {param.requires_grad}")

In [168]:
model = nn.Sequential(nn.Linear(8 ,1))
print(model[0].weight.requires_grad)

True


# Giải quyết Exploding Gradient
## Khởi tạo weight
- ReLU/LeakyReLU/ELU: `nn.init.kaiming_uniform_`
- Tanh/Sigmoid: `nn.init.xavier_uniform_`
- Bias: `nn.init.zeros_(bias)`

## Hàm kích hoạt tốt
- `F.relu()`: Phổ thông, trả về 0 với value âm
- `F.leaky_relu(x, 0.01)`: Trả về epsilon * value (tránh dead neurons)
- **`F.elu(x)`**: Không trả về 0 với value âm, smooth function

## Batch Normalization
- Mục đích: Chuẩn hóa output qua từng lớp
- Công thức: `value_zscore = (value - value.mean()) / value.std()`
- Sau đó scale: `y = value.std() * value_zscore + value.mean()`
- Pattern: `Linear → BatchNorm → Activation`


## Gradient Clipping
- torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

## Dropout Regularization
- self.dropout = nn.Dropout(0.2)

# Sử dụng OOP để định nghĩa Dataset & Model

In [246]:
# Định nghĩa Dataset bằng OOP 
# Dataset + dữ liệu tensor thì dùng luôn TensorDataset cho nhanh
# Không phải có 2 method len & getitem

class WaterDataset(Dataset):
    def __init__(self, csv_path):
        super().__init__()
        df = pd.read_csv(csv_path)
        self.data = np.array(df, dtype=np.float32)


    # Bắt buộc phải tự có 2 phương thức len & getitem
    def __len__(self):
        return self.data.shape[0]
    
    def __getitem__(self, idx):
        features = self.data[idx, : -1]
        label = self.data[idx, -1]
        return features, label
    

train_dataset = WaterDataset(
    'data/water_potability/water_train.csv'
)

test_dataset = WaterDataset(
    'data/water_potability/water_test.csv'
)
train_dataloader = DataLoader(
    train_dataset,
    batch_size=2,
    shuffle=True
)

test_dataloader = DataLoader(
    train_dataset,
    batch_size=2,
    shuffle=False
)

# features, target = next(iter(train_dataloader))
# print(features, target)

In [ ]:
# Định nghĩa Model bằng OOP

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(9, 16, dtype=torch.float32)
        self.fc2 = nn.Linear(16, 8, dtype=torch.float32)
        self.fc3 = nn.Linear(8, 1, dtype=torch.float32)

        self.bn1 = nn.BatchNorm1d(16)
        self.bn2 = nn.BatchNorm1d(8)

        self.dropout = nn.Dropout(0.2)

        self.layer_init()

    def forward(self, x):
        # Layer 1: FC -> BN -> RELU -> DROPOUT
        x = self.fc1(x)
        x = self.bn1(x)
        x = F.relu(x)
        x = self.dropout(x)

        # Layer 2: FC -> BN -> RELU -> DROPOUT
        x = self.fc2(x)
        x = self.bn2(x)
        x = F.relu(x)
        x = self.dropout(x)

        x = F.sigmoid(self.fc3(x))

        return x

    def layer_init(self):
        for layer in self.modules():
            if isinstance(layer, nn.Linear):
                nn.init.kaiming_uniform_(layer.weight)
                nn.init.zeros_(layer.bias)


net = Net()
# for name, param in net.named_parameters():
#     print(f"{name}: min={param.min()}, max={param.max()}")

# Giải thích về 1 loại Optimizers
**SGD**
- Đơn giản, cho mô hình đơn giản
- Hiếm dùng

**Adaptive Gradient - Adagrad**
- Thích hợp cho dữ liệu thưa (9 số 0 & 1 số 1)
- Có thể giảm lr quá nhanh

**Root Mean Square Propagation - RMSprop**
- Update trọng số dựa trên kích thước của gradient lớp trước
- Dùng khi: RNN, time series

**Adam**
- Hay spam thằng này
- RMSprop + Momentum
